### module import protection

In [ ]:
if globals().get('LOADED_SETTINGS') == None:
    display('LOADED_SETTINGS')
    LOADED_SETTINGS=True

### modules

In [ ]:
if globals().get('LOADED_ANNOTATIONS') == None:
    %run ANNOTATIONS.ipynb

In [ ]:
if globals().get('LOADED_PATTERN_OBSERVER') == None:
    %run PATTERN_OBSERVER.ipynb

In [ ]:
if globals().get('VIEWING_TABS') == None:
    %run VIEWING_TABS.ipynb # dropdown needs to know what plotting options are

### imports

In [ ]:
from ipywidgets import VBox,HBox,Text,DatePicker,Button,Layout,Dropdown
from ipywidgets import Color # pretty color for apply button
from pathlib import Path # for parsing directory contents

### begin

In [ ]:
'''
SUBJECT
- CHECKBOXES observes SETTINGS for options
- DATA_CONTAINER observes SETTINGS for repopulation purposes
'''
class SETTINGS(ISubject):
    ####################################
    # static
    ####################################
    _default_backtest_path=r'C:\Users\ahkar\OneDrive\Documents\Data\B3'
    
    _view_types=[
        'Extraday',
        'Intraday',
        'Factor',
    ]
    
    _return_types=[
        'Cash + Future',
        'Cash',
        'Future',
    ]
    
    '''each entry of SETTINGS._option_names needs to be a widget and also instantiable'''
    _option_names=[
        'backtest_path',
        'view_type',
        'plot_type',
        'return_type',
        'date_from',
        'date_to',
    ]

    ####################################
    # constructor
    ####################################
    def __init__(self,
        name='SETTINGS',
        width='25%'
        ):
        self._logging_level   = 1
        self._width           = width
        
        self.name             = name
        self.widget           = self._make_widget()
        
        # for observer pattern
        self.reference        = {}
        self._observers       = set()
        
        # update reference
        self._update_reference()
        
    ####################################
    # observer pattern
    ####################################
    # subject
    def attach(self,observer : IObserver) -> None :
        print('OBSERVER PATTERN',':',observer.name,'OBSERVES',self.name)
        self._observers.add(observer)
        
    def detach(self,observer : IObserver) -> None :
        print('OBSERVER PATTERN',':',observer.name,'STOPS OBSERVING',self.name)
        self._observers.remove(observer)
        
    def notify(self,info) -> None :
        print('OBSERVER PATTERN',':',self.name,'NOTIFIES',len(self._observers),'OBSERVERS')
        
        # print receivers
        for observer in self._observers:
            print('OBSERVER PATTERN',':',self.name,'NOTIFIES',observer.name)
            observer.react(self.name,info)
            
    ####################################
    # button state
    ####################################
    def _button_inactive(self,button : Button) -> None :
        button.style.button_color=Color(None).name
        button.disabled=True
        
    def _button_active(self,button : Button,color : str ='pink') -> None :
        button.style.button_color=color
        button.disabled=False
        
    def _update_button_state(self) -> None :
        if self._logging_level > 0:
            print('_update_button_state','called')
            
        if self.applied_settings==self._current_settings:
            # if nothing changed
            self._button_inactive(self._WIDGET_CONTROLS[0])
            self._button_inactive(self._WIDGET_CONTROLS[1])
        else:
            # if something changed
            self._button_active(self._WIDGET_CONTROLS[0],color='lightgreen')
            self._button_active(self._WIDGET_CONTROLS[1],color='pink')

    ####################################
    # button click
    ####################################
    def _click_apply(self,change) -> None :
        print(self.name,':','CLICKED',change.description)
        self._update_reference() # update reference
        self._update_button_state() # update button state
        self.notify(self.reference) # notify observers

    '''reset to last applied values'''
    def _click_cancel(self,change) -> None :
        print(self.name,':','CLICKED',change.description)
        for widget,value in zip(self._WIDGET_OPTIONS,self.applied_settings): widget.value=value

    def _click_reset(self,change) -> None :
        print(self.name,':','CLICKED',change.description)
        self._WIDGET_OPTIONS[SETTINGS._option_names.index('return_type')].value=SETTINGS._return_types[0]
        self._WIDGET_OPTIONS[SETTINGS._option_names.index('date_from')].value=None
        self._WIDGET_OPTIONS[SETTINGS._option_names.index('date_to')].value=None

    def _click_refresh(self,change) -> None :
        self._click_cancel(change) # forget current settings
        self.notify(self.reference) # notify observers with current state of self.reference

    def _click_reload(self,change) -> None :
        self._click_cancel(change) # forget current settings
        self.notify({'force_reload':True}) # notify observers with `force_reload` message
        
    ####################################
    # widget onchange
    ####################################
    def _onchange_backtest_path(self,change) -> None :
        print(self.name,':','CHANGED',change.owner.description,change.new)
        self._update_button_state() # update button state
        
    def _onchange_view_type(self,change) -> None :
        print(self.name,':','CHANGED',change.owner.description,change.new)
        self._update_button_state() # update button state

    def _onchange_plot_type(self,change) -> None :
        print(self.name,':','CHANGED',change.owner.description,change.new)
        self._update_button_state() # update button state

    def _onchange_return_type(self,change) -> None :
        print(self.name,':','CHANGED',change.owner.description,change.new)
        self._update_button_state() # update button state

    def _onchange_date_from(self,change) -> None :
        print(self.name,':','CHANGED',change.owner.description,change.new)
        self._update_button_state() # update button state

    def _onchange_date_to(self,change) -> None :
        print(self.name,':','CHANGED',change.owner.description,change.new)
        self._update_button_state() # update button state
        
    ####################################
    # build GUI
    ####################################
    def _make_widget(self) -> VBox :
        # controls
        controls=HBox(self._make_controls())
        
        # options
        options=VBox(
            children=self._make_options(),
            layout=Layout(height='300px')
        )
        
        # return
        return VBox(
            children=[controls,options],
            layout=Layout(border='1px solid black',width=self._width,overflow_x='scroll')
        )
    
    def _make_controls(self) -> BUTTONS :
        button_apply=Button(description='apply',disabled=True)
        button_apply.on_click(self._click_apply)

        button_cancel=Button(description='cancel',disabled=True)
        button_cancel.on_click(self._click_cancel)

        button_reset=Button(description='reset')
        button_reset.on_click(self._click_reset)

        button_refresh=Button(description='refresh')
        button_refresh.on_click(self._click_refresh)

        button_reload=Button(description='reload')
        button_reload.on_click(self._click_reload)

        return [
            button_apply,
            button_cancel,
            button_reset,
            button_refresh,
            button_reload,
        ]

    def _make_options(self) -> WIDGETS:
        # backtest_path
        backtest_path=Text(
            description='path',
            value=SETTINGS._default_backtest_path,
            placeholder='what is a placeholder?',
            disabled=False,
            #continuous_update=False,
        )
        backtest_path.observe(self._onchange_backtest_path,names='value')
        
        # view_type
        view_type=Dropdown(description='view type',options=SETTINGS._view_types)
        view_type.observe(self._onchange_view_type,names='value')
        
        # plot_type
        plot_type=Dropdown(description='plot type',options=SINGLE_PLOT._plot_types) #            list('abcd')) TODO
        plot_type.observe(self._onchange_plot_type,names='value')
        
        # return_type
        return_type=Dropdown(description='return type',options=SETTINGS._return_types)
        return_type.observe(self._onchange_return_type,names='value')
        
        # date_from
        date_from=DatePicker(description='date from')
        date_from.observe(self._onchange_date_from,names='value')
        
        # date_to
        date_to=DatePicker(description='date to')
        date_to.observe(self._onchange_date_to,names='value')
            
        #instantiate and return list of widgets / options
        return list(map(eval,SETTINGS._option_names))

    ####################################
    # properties
    ####################################
    @property
    def _WIDGET_CONTROLS(self) -> BUTTONS :
        return self.widget.children[0].children
    
    @property
    def _WIDGET_OPTIONS(self) -> WIDGETS :
        return self.widget.children[1].children
    
    @property
    def _current_settings(self) -> List :
        return [x.value for x in self._WIDGET_OPTIONS]
    
    ####################################
    # functions
    ####################################
    def _applied_setting_by_name(self,name : str):
        return self.applied_settings[SETTINGS._option_names.index(name)]

    def _refresh_all_backtests(self) -> ID_PATHS :
        p=self._applied_setting_by_name('backtest_path')
        return {x.stem:x for x in Path(p).iterdir()}
        
    def _refresh_all_books(self) -> ID_PATHS :
        # TODO update this
        return {x:x for x in ['Trading','Quote','MM2','Hedge','Hit']}

    '''
    # TODO assumes settings are correct, backtest_path can be wrong!
    '''
    def _update_reference(self) -> None:
        # update checkpoint
        self.applied_settings           = self._current_settings

        # populate self.reference
        self.reference['backtest_path'] = self._applied_setting_by_name('backtest_path') # str
        self.reference['view_type']     = self._applied_setting_by_name('view_type')     # str
        self.reference['plot_type']     = self._applied_setting_by_name('plot_type')     # str
        self.reference['return_type']   = self._applied_setting_by_name('return_type')   # str
        self.reference['date_from']     = self._applied_setting_by_name('date_from')     # DateTime
        self.reference['date_to']       = self._applied_setting_by_name('date_to')       # DateTime
        self.reference['all_backtests'] = self._refresh_all_backtests()                  # ID_PATHS
        self.reference['all_books']     = self._refresh_all_books()                      # ID_PATHS
                                                                                        
        # show internals
        self._logging()
        
    def _logging(self) -> None:
        for k,v in self.reference.items():
            print(self.name,':','REFERENCE',':',k,':',v)

In [ ]:
# __file__ exists if notebook called with %run
# e.g. only do example if called directly
try:
    __file__
except NameError:
    # example
    a=SETTINGS()
    display(dir(a))
    display(a.widget)